In [63]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras import layers, models, optimizers
from keras.layers import Input, Conv1D, Dense, Bidirectional, MaxPooling1D, LSTM, TimeDistributed, Flatten, Dropout
from keras.models import Model
from keras.utils import to_categorical
from keras import initializers
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K

import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
def load_data():
    df = pd.read_csv("../dataset/embeddings-ft1-clean.csv", sep=";")
    df = df.drop('id', axis=1)
    all_but_target = list(set(df.columns.values)-set(["target"]))
    X = df[all_but_target]
    y = df[["target"]]
    return X.as_matrix(), pd.get_dummies(y).as_matrix()

In [45]:
X, y = load_data()

/home/joze/other/repos/capsnet/venv/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [46]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [47]:
def reshapeX(X):
    return X.reshape((-1, 300, 1)).astype('float32')
def reshapeY(y):
    return y.reshape((-1, 4, 1))

In [48]:
x_train = reshapeX(x_train)
x_test = reshapeX(x_test)

In [50]:
print(x_train.shape)
print(y_train.shape)

(337850, 300, 1)
(337850, 4)


In [51]:
input_shape = [300, 1]
n_class = 4
n_routing = 3

In [70]:
inp = Input(shape=input_shape)
x = Conv1D(filters=256, kernel_size=10, strides=1, padding='valid', activation='relu')(inp)
x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)
x = Dropout(0.25)(x)
x = Conv1D(filters=256, kernel_size=3, strides=1, padding='valid', activation='relu')(x)
x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)
x = Dropout(0.25)(x)
x = Bidirectional(LSTM(128, return_sequences=True), input_shape=(300, 1))(x)
x = Flatten()(x)
x = Dropout(0.25)(x)
out = Dense(n_class, activation='softmax')(x)

In [71]:
model = Model(inputs=inp, outputs=out)
model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])

In [72]:
model.fit(x_train, y_train, batch_size=500, epochs=10, validation_split=0.2)

Train on 270280 samples, validate on 67570 samples
Epoch 1/10
270280/270280 [==============================] - 1135s 4ms/step - loss: 0.6520 - acc: 0.7454 - val_loss: 0.4630 - val_acc: 0.8325
Epoch 2/10
270280/270280 [==============================] - 1134s 4ms/step - loss: 0.4551 - acc: 0.8349 - val_loss: 0.4141 - val_acc: 0.8511
Epoch 3/10
270280/270280 [==============================] - 1136s 4ms/step - loss: 0.4203 - acc: 0.8483 - val_loss: 0.3972 - val_acc: 0.8565
Epoch 4/10
270280/270280 [==============================] - 1134s 4ms/step - loss: 0.3989 - acc: 0.8560 - val_loss: 0.3769 - val_acc: 0.8653
Epoch 5/10
270280/270280 [==============================] - 1133s 4ms/step - loss: 0.3828 - acc: 0.8619 - val_loss: 0.3629 - val_acc: 0.8705
Epoch 6/10
270280/270280 [==============================] - 1133s 4ms/step - loss: 0.3668 - acc: 0.8679 - val_loss: 0.3532 - val_acc: 0.8745
Epoch 7/10
270280/270280 [==============================] - 1133s 4ms/step - loss: 0.3563 - acc: 0.8725

In [73]:
predictions = model.predict(x_test)

In [74]:
preds = np.argmax(predictions, axis=1)
ytest = np.argmax(y_test, axis=1)

In [75]:
accuracy = accuracy_score(ytest, preds)

In [76]:
accuracy

0.8897268626499177